In [22]:
import mdtraj as md
import matplotlib.pyplot as plt
from numpy import dot
import nglview as nv
import numpy as np

# https://biopython.org/docs/1.74/api/Bio.SVDSuperimposer.html
from Bio.SVDSuperimposer import SVDSuperimposer

# Load H-NS dimers
loc_dimers = './data/dimer_pdbs/'
dimers = md.join([md.load(loc_dimers+f'run{i}.pdb') for i in range(0,16)])

# Load site2 - site2 data
loc_site = './data/'
site2_complex = md.load(loc_site+f'site2-dimer-LAA.pdb')
# site2_complex = md.load(loc_site+f'site21.pdb')


In [23]:
fit_idx = [x for x in range(849,1296)]


In [24]:
view = nv.show_mdtraj(site2_complex)
# print(site2_complex)
traj = site2_complex
topology = traj.topology
resids = [res for res in topology.residues]

# print("site2_A")
# print("site2_B")
# print(resids[29:58])

dimer = dimers[7]
view

view2 = nv.show_mdtraj(dimer)
view2

view

fit_site2_site2_idx = topology.select('resid 0 to 28')

top = dimer.topology
residss = [res for res in top.residues]
cor_res = [residue for residue in top.residues if (residue.index >= 52 and residue.index <= 78)]
# print("complex")
print(cor_res)
idx = top.select('resid 53 to 79')
print(idx)

# print(len(topology.select('resid 0 to 28')))

# print(len(top.select('resid 54 to 80')))

# chains = [res for res in top._chains[0]]

# selection = topology.select("resid 0")
# for idx in selection:
#     print(topology._atoms[idx])

# print("")
# selection2 = top.select("resid 54")
# for idx in selection2:
#     print(top._atoms[idx])

# for res in top.residues:
#     print(res.index, res)
#test



[GLU53, ARG54, THR55, ARG56, LYS57, LEU58, GLN59, GLN60, TYR61, ARG62, GLU63, MET64, LEU65, ILE66, ALA67, ASP68, GLY69, ILE70, ASP71, PRO72, ASN73, GLU74, LEU75, LEU76, ASN77, SER78, LEU79]
[ 849  850  851  852  853  854  855  856  857  858  859  860  861  862
  863  864  865  866  867  868  869  870  871  872  873  874  875  876
  877  878  879  880  881  882  883  884  885  886  887  888  889  890
  891  892  893  894  895  896  897  898  899  900  901  902  903  904
  905  906  907  908  909  910  911  912  913  914  915  916  917  918
  919  920  921  922  923  924  925  926  927  928  929  930  931  932
  933  934  935  936  937  938  939  940  941  942  943  944  945  946
  947  948  949  950  951  952  953  954  955  956  957  958  959  960
  961  962  963  964  965  966  967  968  969  970  971  972  973  974
  975  976  977  978  979  980  981  982  983  984  985  986  987  988
  989  990  991  992  993  994  995  996  997  998  999 1000 1001 1002
 1003 1004 1005 1006 1007 100

In [25]:
top_a = traj.topology
top_b = dimers[7].topology

In [26]:
print(top_a.to_fasta())
print(top_b.to_fasta())
view


['MSEALKILNNIRTLRAQARECTLETLEEMLEKLEVVVNERREEESAAAAEVEERTRKLQQYREMLIADGIDPNELLNSLAAVKSGTKAKRAQRPAKYSYVDENGETKTWTGQGRTPAVIKKAMDEQGKSLDDFLIKQ', 'MSEALKILNNIRTLRAQARECTLETLEEMLEKLEVVVNERREEESAAAAEVEERTRKLQQYREMLIADGIDPNELLNSLAAVKSGTKAKRAQRPAKYSYVDENGETKTWTGQGRTPAVIKKAMDEQGKSLDDFLIKQ']
['MSEALKILNNIRTLRAQARECTLETLEEMLEKLEVVVNERREEESAAAAEVEERTRKLQQYREMLIADGIDPNELLNSLAAVKSGTKAKRAQRPAKYSYVDENGETKTWTGQGRTPAVIKKAMDEQGKSLDDFLIKQ', 'MSEALKILNNIRTLRAQARECTLETLEEMLEKLEVVVNERREEESAAAAEVEERTRKLQQYREMLIADGIDPNELLNSLAAVKSGTKAKRAQRPAKYSYVDENGETKTWTGQGRTPAVIKKAMDEQGKSLDDFLIKQ']


NGLWidget()

In [27]:
view2

NGLWidget()

In [31]:
print(top_a.n_chains)
print(top_a.chain(1))

2


In [32]:
# Extract xyz coordinates of chosen frame for complex and dimer
def extract_coords(traj):
    return traj.xyz[0].T

# Get indices of atoms to fit, in this case only the last 48 residues, which coincide with the DBD
def get_index(molecule, chain_idx, atoms):
    return [at.index for res in molecule.top._chains[chain_idx]._residues[-atoms:] for at in res.atoms]

def slice_transpose(molecule,molecule_idx):
    return molecule.atom_slice(molecule_idx).xyz[0].T

def superimpose(coords_a, coords_b):
    sup = SVDSuperimposer()

    # Set the coords y will be rotated and translated on x
    x = coords_a.T
    y = coords_b.T
    sup.set(x, y)

    # Do the lsq fit
    sup.run()

    # Get the rms
    rms = sup.get_rms()

    # Get rotation (right multiplying!) and the translation
    rot, tran = sup.get_rotran()

    # Rotate y on x
    y_on_x1 = dot(y, rot) + tran

    # Same thing
    y_on_x2 = sup.get_transformed()
    
    return rms, rot, tran

In [33]:
def fit(struct_a, struct_b, fit_idx_a, fit_idx_b):
    
    # van fit selection een lijst van indeces
    xyz_a = extract_coords(struct_a)
    xyz_b = extract_coords(struct_b)
    
    atom_set_a = slice_transpose(struct_a, fit_idx_a)
    atom_set_b = slice_transpose(struct_b, fit_idx_b)
    
    rms, rot, tran = superimpose(atom_set_a, atom_set_b)
    
    a = xyz_a.T
    b = xyz_b.T

    # rotate b on a
    b_on_a1 = dot(b, rot) + tran

    # replace coordinates 
    struct_a.xyz = b_on_a1
    
    fitted = struct_b.stack(struct_a)
    
    selection = [at.index for idx,chain in enumerate(fitted.top.chains) if idx != 2 for at in chain.atoms]
    # probably easiest to use atomslice to remove/delete monomer
    final = fitted.atom_slice(selection)
    
    # return final
    return final, rms

In [35]:
#resid 53 - 79 = atom 849 - 1295

test, rms = fit(traj, dimers[7], idx, idx)

view = nv.show_mdtraj(test)
view

ValueError: xyz must be shape (Any, 2174, 3). You supplied  (1, 4390, 3)